In [277]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [338]:
pkg_cnt = [1000, 1000, 1000]
cargo_distribution = [.4, .5, .1]
weight = [43, 48, 7.1]

container_dim = [11800, 2340, 2690]
volume = np.prod(container_dim, dtype = "double")
weight_limit = 22_000

space_occupancy = .8

c1_dim1 = [605, 470, 500]
c1_dim2 = [470, 605, 500]
c1_vol = np.prod(c1_dim1)
c2_dim1 = [675, 490, 510]
c2_dim2 = [490, 675, 510]
c2_vol = np.prod(c2_dim1)
c3_dim1 = [1150, 290, 230]
c3_dim2 = [290, 1150, 230]
c3_vol = np.prod(c3_dim1)

box_dims = np.array([c1_dim1, c2_dim1, c3_dim1])

In [339]:
def vol_limit(container_dim = container_dim, cont_dims = cont_dims, c1_dim1 = c1_dim1, c2_dim2 = c2_dim2, c3_dim1 = c3_dim1, occupancy = .9,
             weight_limit = 22_000):
    pkg_cnt = cargo_count()
#     for j in np.arange(np.sum(pkg_cnt)):
    return pkg_cnt

In [340]:
def cargo_count(container_dim = container_dim, box_dims = box_dims, occupancy = space_occupancy,
               cargo_distribution = cargo_distribution):
    cont_vol = np.prod(container_dim, dtype = "double")
    volumes = list()
    for box in box_dims:
        volumes.append(np.prod(box))
    cont_vol_occ = cont_vol * occupancy
    vol_allocation = np.array(cargo_distribution) * cont_vol_occ
    cargo_count = np.divide(vol_allocation, volumes)
    return np.trunc(cargo_count)

In [341]:
num_boxes = cargo_count()
num_boxes

array([167., 176.,  77.])

In [342]:
class Point():
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
    @classmethod
    def two_dim_point(cls, x, y):
        return cls(x, y, 0)
    
    def __str__(self):
        return "(" + str(self.x) + "," + str(self.y) + "," + str(self.z) + ")"
    
    def __gt__(self, Point2):
        print(np.linalg.norm([self.x, self.y, self.z]) > np.linalg.norm([self.x, self.y, self.z]))
        return np.linalg.norm([self.x, self.y, self.z]) < np.linalg.norm([self.x, self.y, self.z])
    

In [343]:
class Box():
    def __init__(self, btype, orientation, dims):
        self.btype = btype
        if orientation == 1:
            hold = dims[0]
            dims[0] = dims[1]
            dims[1] = hold
        self.orientation = orientation
        self.dims = dims

In [ ]:
# corner: list of corner points. (Use Point object defined above)
# space: No clue. It's just there. Don't question it.
# box_count: number of boxes in container.
# location of boxes: Set as 

class Container(object):
    def __init__(self, corner, space, box_count, box_loc, dimension):
        self.corner = corner
        self.space = space
        self.box_count = box_count
        self.box_loc = box_loc
        self.dimension = dimension
        
    @classmethod
    def empty(cls, dimension):
        return cls([], 0, 0, {}, dimension)
    
    # box_type: the box number of the box_type you are placing in box
    # orientation: orientation number corresponding to the following convention:
        # 0: (L, W, H) Note: This is the original input sequence.
        # 1: (W, L, H)
    # corner: An item of Point class with field values set for the x, y, and z values.
    # dims: A list with the dimension of boxes. 
    # cargo: box object
    
    #This class method acts to "place" a box in the container.    
    def place(self, corner, cargo):
        
        dictionary = self.box_loc.items()
        overlap = 0
#         for corn, val in dictionary:
#             btype, x, y, z = val
#             if((corner.x < x && corner.y < y && corner.z < z) && )
        self.box_loc[corner] = (cargo.btype, corner.x + cargo.dims[0], corner.y + cargo.dims[1], 
                           corner.z + cargo.dims[2])
        self.box_count += 1
        

    
    def order_boxes(self):
        pairs = self.box_loc.items()
        
        return sorted(pairs, key = lambda x: (x[1][2], x[1][1]), reverse = True)
            
        
    # box_list: List of boxes not yet added to container
    
    def sort_corners(self):
        return self.corner.sort()
    
    def find_corners_2d(self, box_list, box_dims):
        self.corner = []
        if self.box_count == 0:
            self.corner.append(Point.two_dim_point(0, 0))
            return
        #Phase 1: Identify Extreme Points.
        
        x_bar = 0
        m = 0
        em = -1
        ordered_boxes = self.order_boxes()
        
        for index, value in enumerate(ordered_boxes):
            corner, b_data = value
            btype, x, y, z = b_data
            if x >= x_bar:
                m += 1
                em = index
                x_bar = x
                
        # Phase 2: Find Corner Points
        self.corner.append(Point.two_dim_point(0, ordered_boxes[0][1][2]))
        self.corner.append(Point.two_dim_point(ordered_boxes[em][1][1], 0))
        for j in np.arange(1, m):
            self.corner.append(Point.two_dim_point(ordered_boxes[j-1][1][1], ordered_boxes[j][1][2]))
            
        # Phase 3: Remove redundant/infeasible corner points:
        present = []
        
        for i in np.arange(len(box_list)):
            if box_list[i] != 0:
                present.append(box_dims[i])
        present = np.array(present)
                
        x_avail = present[:, 0]
        y_avail = present[:, 1]
        
        print("minimum x: " + str(x_avail))
        
        for corn in self.corner:
            if (corn.x + min(x_avail) > self.dimension[0]) or (corn.y + min(y_avail) > self.dimension[1]):
                self.corner.remove(corn)

# Testing

In [345]:
cont_2d = Container.empty([11800, 2340, 2690])

In [286]:
cargo = Box(0, 0, box_dims[0])
cargo.dims
cargo1 = Box(0, 0, box_dims[0])

In [287]:
cont_2d.place(Point.two_dim_point(0,0), cargo)

cont_2d.box_loc

cont_2d.find_corners_2d(num_boxes, box_dims)

print(cont_2d.corner)

[<__main__.Point object at 0x000002B8AFACABB0>, <__main__.Point object at 0x000002B8AFACABE0>]


In [288]:
for i in cont_2d.corner:
    print(i)

(0,470,0)
(605,0,0)


In [289]:
cont_2d.place(cont_2d.corner[0], cargo1)
cont_2d.find_corners_2d(num_boxes, box_dims)
for i in cont_2d.corner:
    print(i)

(0,940,0)
(605,0,0)
(605,470,0)


In [290]:
cont_2d.place(cont_2d.corner[1], cargo1)
cont_2d.find_corners_2d(num_boxes, box_dims)
for i in cont_2d.corner:
    print(i)

cont_2d.sort_corners()

for i in cont_2d.corner:
    print(i)

(0,940,0)
(1210,0,0)
(605,470,0)
False
False
(0,940,0)
(1210,0,0)
(605,470,0)


In [291]:
cont_2d.find_corners_2d(num_boxes, box_dims)

print(cont_2d.corner)

[<__main__.Point object at 0x000002B8AFACAB20>, <__main__.Point object at 0x000002B8AFACAD90>, <__main__.Point object at 0x000002B8AFABA580>]


In [292]:
for i in cont_2d.corner:
    print(i)

(0,940,0)
(1210,0,0)
(605,470,0)


In [293]:
cont_test = Container.empty([1, 1, 1])
cont_test.find_corners_2d(num_boxes, box_dims)
print(cont_test.corner[0])

(0,0,0)


In [346]:
cont_2d.place(Point.two_dim_point(0, 0), cargo)
cont_2d.place(Point.two_dim_point(0, 470), cargo)
cont_2d.place(Point.two_dim_point(0, 940), cargo)
cont_2d.place(Point.two_dim_point(605, 0), cargo)
cont_2d.place(Point.two_dim_point(1210, 0), cargo)
cont_2d.place(Point.two_dim_point(605, 470), cargo)

In [347]:
cont_2d.find_corners_2d(num_boxes, box_dims)
for i in cont_2d.corner:
    print(i)

cont_2d.sort_corners()

for i in cont_2d.corner:
    print(i)

minimum x: [ 605  675 1150]
(0,1410,0)
(1815,0,0)
(605,940,0)
(1210,940,0)
False
False
False
(0,1410,0)
(1815,0,0)
(605,940,0)
(1210,940,0)


In [296]:
test = np.arange(9).reshape(3, -1)
test

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [332]:
type(box_dims)

list

In [337]:
box_dims[:,0]

array([ 605,  675, 1150])

In [298]:
arr =[1,0,1]

In [335]:
test[:,0]

array([0, 3, 6])

In [300]:
test

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [301]:
np.arange(2,2)

array([], dtype=int32)

In [302]:
cont_2d = Container.empty([11800, 2340, 2690])
for i in np.arange(num_boxes[0]):
    cargo = Box(0, 0, box_dims[0])
#     cont_2d.sort_corners()
    cont_2d.find_corners_2d(num_boxes, box_dims)
    print(cont_2d.corner)
    cont_2d.place(cont_2d.corner[0], cargo)
    num_boxes[0] -= 1

[<__main__.Point object at 0x000002B8AFACA6A0>, <__main__.Point object at 0x000002B8AF875EB0>]
[<__main__.Point object at 0x000002B8AF875EB0>, <__main__.Point object at 0x000002B8AF875A30>, <__main__.Point object at 0x000002B8AF83CFA0>]
[<__main__.Point object at 0x000002B8AF875A30>, <__main__.Point object at 0x000002B8AFADEEE0>, <__main__.Point object at 0x000002B8AF83CC40>, <__main__.Point object at 0x000002B8AF83C8E0>]
[<__main__.Point object at 0x000002B8AFADEEE0>, <__main__.Point object at 0x000002B8AF875A60>, <__main__.Point object at 0x000002B8AF83CE50>, <__main__.Point object at 0x000002B8AF83CCA0>, <__main__.Point object at 0x000002B8AF83C9A0>]
[<__main__.Point object at 0x000002B8AF875A60>, <__main__.Point object at 0x000002B8AFADE550>, <__main__.Point object at 0x000002B8AF83C8E0>, <__main__.Point object at 0x000002B8AF83CE80>, <__main__.Point object at 0x000002B8AF83CD60>, <__main__.Point object at 0x000002B8AF83C3D0>]
[<__main__.Point object at 0x000002B8AFADE550>, <__main

[<__main__.Point object at 0x000002B8AFAC70D0>, <__main__.Point object at 0x000002B8AFAC7130>, <__main__.Point object at 0x000002B8AFAC7580>, <__main__.Point object at 0x000002B8AFAC7460>, <__main__.Point object at 0x000002B8AFAC7BB0>, <__main__.Point object at 0x000002B8AFAC7F70>, <__main__.Point object at 0x000002B8AFAC7610>, <__main__.Point object at 0x000002B8AFAC7790>, <__main__.Point object at 0x000002B8AFAC73D0>, <__main__.Point object at 0x000002B8AFAC7100>, <__main__.Point object at 0x000002B8AFAEE430>, <__main__.Point object at 0x000002B8AFAEED30>, <__main__.Point object at 0x000002B8AFAEEC40>, <__main__.Point object at 0x000002B8AFAEEA60>, <__main__.Point object at 0x000002B8AFAEECA0>, <__main__.Point object at 0x000002B8AFAEEEB0>, <__main__.Point object at 0x000002B8AFAEEF70>, <__main__.Point object at 0x000002B8AFAEEA00>, <__main__.Point object at 0x000002B8AFAEEDF0>, <__main__.Point object at 0x000002B8AFAEE910>, <__main__.Point object at 0x000002B8AFAEE820>, <__main__.Po

[<__main__.Point object at 0x000002B8AFAEED00>, <__main__.Point object at 0x000002B8AFAEEB20>, <__main__.Point object at 0x000002B8AFAEE820>, <__main__.Point object at 0x000002B8AFAEE880>, <__main__.Point object at 0x000002B8AF96C2E0>, <__main__.Point object at 0x000002B8AF96C640>, <__main__.Point object at 0x000002B8AF96C970>, <__main__.Point object at 0x000002B8AF96CA90>, <__main__.Point object at 0x000002B8AF96CC10>, <__main__.Point object at 0x000002B8AF96CD60>, <__main__.Point object at 0x000002B8AF96CE80>, <__main__.Point object at 0x000002B8AF96C220>, <__main__.Point object at 0x000002B8AF96C160>, <__main__.Point object at 0x000002B8AF96C670>, <__main__.Point object at 0x000002B8AF96CF10>, <__main__.Point object at 0x000002B8AF96CCA0>, <__main__.Point object at 0x000002B8AF96CC40>, <__main__.Point object at 0x000002B8AF96CCD0>, <__main__.Point object at 0x000002B8AF96C490>, <__main__.Point object at 0x000002B8AF96C550>, <__main__.Point object at 0x000002B8AF96C130>, <__main__.Po

[<__main__.Point object at 0x000002B8AFAE12B0>, <__main__.Point object at 0x000002B8AFAE1B50>, <__main__.Point object at 0x000002B8AFAE1B20>, <__main__.Point object at 0x000002B8AFAE1A00>, <__main__.Point object at 0x000002B8AFAE18B0>, <__main__.Point object at 0x000002B8AFAE1700>, <__main__.Point object at 0x000002B8AFAE1E50>, <__main__.Point object at 0x000002B8AFAE1550>, <__main__.Point object at 0x000002B8AFAE19A0>, <__main__.Point object at 0x000002B8AFAE12E0>, <__main__.Point object at 0x000002B8AFAE1A60>, <__main__.Point object at 0x000002B8AFAE1880>, <__main__.Point object at 0x000002B8AFAE1490>, <__main__.Point object at 0x000002B8AFAE1220>, <__main__.Point object at 0x000002B8AFAE15B0>, <__main__.Point object at 0x000002B8AFAE1610>, <__main__.Point object at 0x000002B8AFAE1250>, <__main__.Point object at 0x000002B8AFAE1640>, <__main__.Point object at 0x000002B8AFAEEC70>, <__main__.Point object at 0x000002B8AFAC1400>, <__main__.Point object at 0x000002B8AFAC1610>, <__main__.Po

In [303]:
for i in cont_2d.corner:
    print(i)

(0,78020,0)
(605,0,0)
(605,77550,0)
(605,77080,0)
(605,76610,0)
(605,76140,0)
(605,75670,0)
(605,75200,0)
(605,74730,0)
(605,74260,0)
(605,73790,0)
(605,73320,0)
(605,72850,0)
(605,72380,0)
(605,71910,0)
(605,71440,0)
(605,70970,0)
(605,70500,0)
(605,70030,0)
(605,69560,0)
(605,69090,0)
(605,68620,0)
(605,68150,0)
(605,67680,0)
(605,67210,0)
(605,66740,0)
(605,66270,0)
(605,65800,0)
(605,65330,0)
(605,64860,0)
(605,64390,0)
(605,63920,0)
(605,63450,0)
(605,62980,0)
(605,62510,0)
(605,62040,0)
(605,61570,0)
(605,61100,0)
(605,60630,0)
(605,60160,0)
(605,59690,0)
(605,59220,0)
(605,58750,0)
(605,58280,0)
(605,57810,0)
(605,57340,0)
(605,56870,0)
(605,56400,0)
(605,55930,0)
(605,55460,0)
(605,54990,0)
(605,54520,0)
(605,54050,0)
(605,53580,0)
(605,53110,0)
(605,52640,0)
(605,52170,0)
(605,51700,0)
(605,51230,0)
(605,50760,0)
(605,50290,0)
(605,49820,0)
(605,49350,0)
(605,48880,0)
(605,48410,0)
(605,47940,0)
(605,47470,0)
(605,47000,0)
(605,46530,0)
(605,46060,0)
(605,45590,0)
(605,45120,0

In [304]:
cont_2d.box_count

167

In [305]:
num_boxes

array([  0., 176.,  77.])